In [1]:
from ner_pipeline import *

In [2]:
import os
os.chdir('..')

In [2]:
import seqeval.metrics as sqm

In [3]:
!pip freeze

absl-py==0.14.1
aiohttp==3.7.4.post0
aioredis==2.0.1
altair==4.1.0
appnope==0.1.2
APScheduler==3.6.3
argon2-cffi==21.1.0
arviz==0.11.2
astor==0.8.1
astunparse==1.6.3
async-timeout==3.0.1
attrs==21.2.0
backcall==0.2.0
backports.entry-points-selectable==1.1.0
base58==2.1.0
beautifulsoup4==4.10.0
bleach==4.1.0
blinker==1.4
blis==0.7.4
Boruta==0.3
Brotli==1.0.9
cachetools==4.2.2
catalogue==2.0.6
certifi==2021.10.8
cffi==1.15.0
cftime==1.5.1
chardet==4.0.0
charset-normalizer==2.0.6
clarifai==2.6.2
click==7.1.2
cloudpickle==2.0.0
codetiming==1.3.0
configparser==3.8.1
cycler==0.10.0
cymem==2.0.5
dash==2.0.0
dash-core-components==2.0.0
dash-html-components==2.0.0
dash-table==5.0.0
datasets==1.13.3
debugpy==1.4.3
decorator==5.1.0
defusedxml==0.7.1
delayed==0.11.0b1
dill==0.3.4
distlib==0.3.3
docker-pycreds==0.4.0
EasyProcess==0.3
emoji==1.5.0
en-core-web-sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.1.0/en_core_web_sm-3.1.0-py3-none-any.whl
entrypoints==0.3
e

In [28]:
dir(sqm)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'accuracy_score',
 'classification_report',
 'f1_score',
 'performance_measure',
 'precision_score',
 'recall_score',
 'sequence_labeling',
 'v1']

In [23]:
str(getattr(P, 'model').bilstm)

'LSTM(300, 5, num_layers=2, batch_first=True, bidirectional=True)'

In [19]:
getattr(P, 'model')

BiLSTMCRF(
  (word_embedding): Embedding(400005, 300)
  (bilstm): LSTM(300, 5, num_layers=2, batch_first=True, bidirectional=True)
  (hidden2tag): Linear(in_features=10, out_features=256, bias=True)
  (crf): CRF(
    (fc): Linear(in_features=10, out_features=64, bias=True)
  )
)

In [3]:
P = NER_AND_AL_Pipeline(training_data_path='db/annotations/annotations.csv')
P.word_embedding()
P.build_bilstm_crf()
a,b,c = P.predict_eval()

Training set size:  1134 , Testing set size:  201
# non aligned  80


 1/1 loss:  8.99: 100%|██████████| 33/33 [03:37<00:00,  6.59s/it]


{0: 'B-DATASET_NAME', 1: 'I-DATASET_NAME', 2: 'L-DATASET_NAME', 3: 'U-DATASET_NAME', 4: 'O', 5: '[PAD]', 6: '[CLS]', 7: '[SEP]', 8: 'X'}
{0: 'B-DATASET_NAME', 1: 'I-DATASET_NAME', 2: 'L-DATASET_NAME', 3: 'U-DATASET_NAME', 4: 'O', 5: '[PAD]', 6: '[CLS]', 7: '[SEP]', 8: 'X'}


/Users/spezzata/opt/anaconda3/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: L-DATASET_NAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/spezzata/opt/anaconda3/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: U-DATASET_NAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/spezzata/opt/anaconda3/lib/python3.7/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification report(Entity level):
              precision    recall  f1-score   support

DATASET_NAME       0.00      0.00      0.00       140

   micro avg       0.00      0.00      0.00       140
   macro avg       0.00      0.00      0.00       140
weighted avg       0.00      0.00      0.00       140

Sentence-level Accuracy: 0.32558139534883723


In [4]:
a

0.0

In [5]:
b

0.32558139534883723

In [11]:
c['precision']

TypeError: string indices must be integers

In [4]:
dat = pd.read_csv('db/annotations/annotations.csv')
unnanoted_text = dat.head(150)['text']
unnatoted_label = [0 for i in range(150)]

In [5]:
P.active_learning(unannotated_texts=pd.Series(unnanoted_text), unannotated_labels=np.array(unnatoted_label), ret_idx=True)

100%|██████████| 1/1 [00:01<00:00,  1.33s/it]

{0: 'B-DATASET_NAME', 1: 'I-DATASET_NAME', 2: 'L-DATASET_NAME', 3: 'U-DATASET_NAME', 4: 'O', 5: '[PAD]', 6: '[CLS]', 7: '[SEP]', 8: 'X'}


array([ 21, 122,  15])

In [6]:
from utils.db_connection import *

In [10]:
database = r"db/sdg_data_catalog_test.db"
def extract_all_paragraphs(database=database):
        conn = create_connection(database)
        cur = conn.cursor()
        cur.execute('''select paragraph_id, body_text, paper_id from paragraph''')
        res = cur.fetchall()
        return res
dat = pd.DataFrame(extract_all_paragraphs(database=database), columns=['paragraph_id', 'body_text', 'paper_id'])

In [15]:
list(dat.values)[0][0]

'0fde93da-ff1b-11eb-8481-acde48001122'

In [9]:
P.eval(unannotated_texts=pd.Series(unnanoted_text), unannotated_labels=np.array(unnatoted_label))

100%|██████████| 1/1 [00:00<00:00,  2.37it/s]

{0: 'B-DATASET_NAME', 1: 'I-DATASET_NAME', 2: 'L-DATASET_NAME', 3: 'U-DATASET_NAME', 4: 'O', 5: '[PAD]', 6: '[CLS]', 7: '[SEP]', 8: 'X'}


(array([ 167.25068665,  182.99395752, 1051.12524414, 1235.22143555,
        1939.10498047,   70.14147949,   61.92136765,  507.92919922,
        1083.4987793 ,  587.76873779,  661.96661377, 1042.66711426,
          57.92435074,   86.14194489,  134.80749512,  116.7381134 ,
        1083.54577637,  256.48269653,  369.83380127,   68.56550598,
         151.21174622,  108.62971497,  743.03851318,  118.58599091,
         726.55877686,  434.91522217,  159.19665527,  418.48538208,
         313.35385132,  118.78353882,  199.73260498, 1488.81762695,
         959.01599121,  767.09234619, 1618.51257324,  969.87646484,
         969.64038086,   69.35522461,  751.13867188,  126.89027405,
         159.31620789,   78.21652222, 1772.61169434,  726.82446289,
         151.21292114,  280.93051147,  840.32623291,  102.56558228,
         605.21600342,  142.98149109,  297.14221191,   78.20314026,
          93.63243103,  402.4934082 ,  629.53753662,   76.20092773,
         280.92736816,  491.06491089, 2072.61743

In [10]:
d_word_id = getattr(P, 'd_word_id')
max_len = getattr(P, 'max_len')

In [11]:
model = getattr(P, 'model')

In [12]:
tags = getattr(P, 'tags')
max_seq_len = getattr(P, 'config')["WORD2VEC"]["max_seq_len"]

In [13]:
unannotated_texts_vec = list(unnanoted_text.apply(lambda x: sentences_to_vec(x, d_word_id=d_word_id, max_len=max_len)).values)
unannotated_texts_vec = np.array(unannotated_texts_vec)

unannotated_texts = torch.from_numpy(unannotated_texts_vec).int()
unannotated_labels = torch.from_numpy(np.array(unnatoted_label)).int()
eval_dl = DataLoader(TensorDataset(unannotated_texts, unannotated_labels), 256, shuffle=False)
model.eval()
scores, tag_seq_l, probs, tag_seq_str = [], [], [], []
with torch.no_grad():
            bar = tqdm(eval_dl)
            for bi,(xs,ys) in enumerate(bar):
                score, tag_seq, prob = model(xs)
                score, prob = score.cpu().detach().numpy(), prob.cpu().detach().numpy()
                tag_seq_l.extend(tag_seq)
                scores.extend(score.tolist())
                probs.extend(prob.tolist())
                tag_seq_str.extend(vec_to_tags(tags, tag_seq, max_seq_len))
scores = np.array(scores)
probs = np.array(probs)

100%|██████████| 1/1 [00:00<00:00,  3.22it/s]

{0: 'B-DATASET_NAME', 1: 'I-DATASET_NAME', 2: 'L-DATASET_NAME', 3: 'U-DATASET_NAME', 4: 'O', 5: '[PAD]', 6: '[CLS]', 7: '[SEP]', 8: 'X'}


In [16]:
strategy_mapping = {
            "RANDOM": RandomStrategy,
            "LC": LeastConfidenceStrategy,
            "NLC": NormalizedLeastConfidenceStrategy,
            "LTP": LeastTokenProbabilityStrategy,
            "MTP": MinimumTokenProbabilityStrategy,
            "MTE": MaximumTokenEntropyStrategy,
            "LONG": LongStrategy,
            "TE": TokenEntropyStrategy,
        }
strategy = 'LTP'
strategy_name = strategy.lower()
max_seq_len = 256
#choice_number = int(datapool.get_total_number() * query_batch_fraction) ## == nb of annotated data + nb of unannotated data
choice_number = int(len(unannotated_texts) * .05)
strategy = strategy_mapping[strategy]

#start an active learning iteration
#self.build_bilstm_crf()
entity_f1_score,sentence_ac_score = P.predict_eval() 
scores, tag_seq, probs, tag_seq_str = P.eval(unannotated_texts=pd.Series(unnanoted_text), unannotated_labels=np.array(unnatoted_label))
idx = strategy.select_idx(choices_number=choice_number, probs=probs, scores=scores, best_path=tag_seq)

{0: 'B-DATASET_NAME', 1: 'I-DATASET_NAME', 2: 'L-DATASET_NAME', 3: 'U-DATASET_NAME', 4: 'O', 5: '[PAD]', 6: '[CLS]', 7: '[SEP]', 8: 'X'}
{0: 'B-DATASET_NAME', 1: 'I-DATASET_NAME', 2: 'L-DATASET_NAME', 3: 'U-DATASET_NAME', 4: 'O', 5: '[PAD]', 6: '[CLS]', 7: '[SEP]', 8: 'X'}


/Users/spezzata/opt/anaconda3/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: L-DATASET_NAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/spezzata/opt/anaconda3/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: U-DATASET_NAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/spezzata/opt/anaconda3/lib/python3.7/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification report(Entity level):
              precision    recall  f1-score   support

DATASET_NAME       0.00      0.00      0.00       140

   micro avg       0.00      0.00      0.00       140
   macro avg       0.00      0.00      0.00       140
weighted avg       0.00      0.00      0.00       140

Sentence-level Accuracy: 0.32558139534883723


100%|██████████| 1/1 [00:00<00:00,  1.33it/s]

{0: 'B-DATASET_NAME', 1: 'I-DATASET_NAME', 2: 'L-DATASET_NAME', 3: 'U-DATASET_NAME', 4: 'O', 5: '[PAD]', 6: '[CLS]', 7: '[SEP]', 8: 'X'}


In [17]:
idx

array([ 21, 122,  32, 147,  15,  55,   1])

In [ ]:
datapool = DataPool(annotated_text=annotated_text, annotated_labels=annotated_labels, 
                            unannotated_texts=unannotated_texts, unannotated_labels=unannotated_labels)
strategy = self.config["ActiveStrategy"]["strategy"]
strategy_name = strategy.lower()
stop_echo = self.config.param["ActiveStrategy"]["stop_echo"]
query_batch_fraction = self.config["ActiveStrategy"]["query_batch_fraction"]
max_seq_len = self.config["WORD2VEC"]["max_seq_len"]
choice_number = int(datapool.get_total_number() * query_batch_fraction) ## == nb of annotated data + nb of unannotated data
strategy = self.strategy[strategy]
for i in range(0, stop_echo):
            #logging.info(
            #    f"[No. {i + 1}/{stop_echo}] ActiveStrategy:{strategy}, BatchFraction: {query_batch_fraction}\n")
            self.build_bilstm_crf()
            entity_f1_score,sentence_ac_score = self.predict_eval() 
            scores, tag_seq, probs, tag_seq_str = self.eval(unannotated_texts=unannotated_texts, unannotated_labels=unannotated_labels)
            idx = strategy.select_idx(choices_number=choice_number, probs=probs, scores=scores, best_path=tag_seq)
            selected_samples = unannotated_labels[idx]
            selected_samples = vec_to_tags(self.tags, selected_samples.tolist(), max_seq_len)
            tag_seq_str = [tag_seq_str[id] for id in idx]
            # update datapool
            self.datapool.update(mode="internal_exchange_u2a", selected_idx=idx)
            _reading_cost = SampleMetrics._reading_cost(selected_samples)
            self.logger.info(f"Reading Cost is {_reading_cost}")
            _annotation_cost = SampleMetrics._annotation_cost(selected_samples, tag_seq_str)
            self.logger.info(f"Annotation Cost is {_annotation_cost}")
            _wrong_select = SampleMetrics._percentage_wrong_selection(selected_samples, tag_seq_str)
            self.logger.info(f"Wrong Selected percentage: {_wrong_select}")
            self.logger.info(f"{strategy_name},{i},{entity_f1_score},{sentence_ac_score},{_reading_cost},{_annotation_cost},{_wrong_select}")
            del self.model
            torch.cuda.empty_cache()

In [18]:
probs

array([[[ 1.93005657, -2.4867053 ,  1.76183736, ..., -7.09553432,
         -6.28738976, -6.8302536 ],
        [ 2.23990846, -2.70045424,  1.84407496, ..., -8.02427673,
         -7.13049078, -7.75085974],
        [ 2.3260138 , -2.70252204,  1.85606396, ..., -8.23274231,
         -7.32636404, -7.96085596],
        ...,
        [ 0.05089406, -0.28436965, -0.04785607, ..., -0.82989633,
         -0.87371463, -0.67716932],
        [ 0.05089406, -0.28436965, -0.04785607, ..., -0.82989633,
         -0.87371463, -0.67716932],
        [ 0.05089406, -0.28436965, -0.04785607, ..., -0.82989633,
         -0.87371463, -0.67716932]],

       [[ 1.88806224, -2.52714276,  1.76253629, ..., -7.03460741,
         -6.22378302, -6.76752615],
        [ 1.94340932, -2.91993618,  1.85125697, ..., -7.56397343,
         -6.62328243, -7.30543947],
        [ 2.33205867, -2.71161318,  1.85739231, ..., -8.24893284,
         -7.34568977, -7.97286844],
        ...,
        [ 0.05089406, -0.28436965, -0.04785607, ..., -